In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import euclidean

In [2]:
df = pd.read_csv("audit_10%.csv")

In [3]:
df

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,NaN,0.500,6.68,NaN,0.2,...,0.4,NaN,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,NaN,0.000,4.83,NaN,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,NaN,0.0,NaN,1.416,NaN,NaN,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,NaN,NaN,0.2,NaN,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,NaN,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,NaN,0.074,NaN,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,NaN,0.0
773,55.57,0.24,0.2,NaN,NaN,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,NaN,0.040,0.00,0.2,0.000,0.20,5.0,NaN,...,0.4,NaN,0.2,0.0,2.0,NaN,0.4,0.5,0.2880,0.0


In [4]:
df.dtypes

Sector_score      float64
PARA_A            float64
Score_A           float64
Risk_A            float64
PARA_B            float64
Score_B           float64
Risk_B            float64
TOTAL             float64
numbers           float64
Score_B.1         float64
Risk_C            float64
Money_Value       float64
Score_MV          float64
Risk_D            float64
District_Loss     float64
PROB              float64
RiSk_E            float64
History           float64
Prob              float64
Risk_F            float64
Score             float64
Inherent_Risk     float64
CONTROL_RISK      float64
Detection_Risk    float64
Audit_Risk        float64
Risk              float64
dtype: object

In [5]:
df.isnull().sum()

Sector_score      74
PARA_A            62
Score_A           76
Risk_A            87
PARA_B            77
Score_B           83
Risk_B            93
TOTAL             94
numbers           88
Score_B.1         94
Risk_C            91
Money_Value       71
Score_MV          84
Risk_D            71
District_Loss     80
PROB              63
RiSk_E            93
History           73
Prob              89
Risk_F            80
Score             75
Inherent_Risk     71
CONTROL_RISK      92
Detection_Risk    75
Audit_Risk        81
Risk               0
dtype: int64

In [6]:
null_counts = df.isnull().sum().sum()
total = df.size
null_percentages = 100 * null_counts / total
print("total nilai :",total)
print("nilai hilang :",null_counts)
print("persentase :",null_percentages)

total nilai : 20176
nilai hilang : 2017
persentase : 9.997026169706581


In [7]:
null_counts = df.isnull().sum()
null_percentages = 100 * null_counts / len(df)
null_percentages

Sector_score       9.536082
PARA_A             7.989691
Score_A            9.793814
Risk_A            11.211340
PARA_B             9.922680
Score_B           10.695876
Risk_B            11.984536
TOTAL             12.113402
numbers           11.340206
Score_B.1         12.113402
Risk_C            11.726804
Money_Value        9.149485
Score_MV          10.824742
Risk_D             9.149485
District_Loss     10.309278
PROB               8.118557
RiSk_E            11.984536
History            9.407216
Prob              11.469072
Risk_F            10.309278
Score              9.664948
Inherent_Risk      9.149485
CONTROL_RISK      11.855670
Detection_Risk     9.664948
Audit_Risk        10.438144
Risk               0.000000
dtype: float64

# Imputasi

In [8]:
def jarak(df2,col):
    euc_dist[col] = []
    euc_dist_append =euc_dist[col].append
    for row in range(df2.shape[0]):
        current = df2.index[df2.iloc[:,col] == 0].tolist()
        current = current[0]
        a = df2.iloc[current,col+1:].tolist()
        b = df2.iloc[current,:col].tolist()
        x1 = b+a
        if(row is not current):
            a=df2.iloc[row,col+1:].tolist()
            b=df2.iloc[row,:col].tolist()
            x2=b+a
        else:
            continue
        x1 = np.nan_to_num(x1)  # Mengganti nilai NaN dengan 0
        x2 = np.nan_to_num(x2)  # Mengganti nilai NaN dengan 0
        dist = euclidean(x1,x2)
        euc_dist_append([dist,df2.iloc[row,col]])
#         print(" x1 = ",x1,"\n x2 = ",x2,"\n distance = ",dist)

    lst = list(euc_dist[col])
    lst.sort()
    euc_dist[col] = lst
#     print(euc_dist[col])

In [9]:
from collections import Counter

def hitung(val, k, temp=None, i=0, counter=0):
    counts = Counter(val)
    total_count = len(counts)
#     print("jumlah val:", total_count)
    if counter >= int((k/2)+1) and total_count <= 2 :
        print("median : ",int((k/2)+1))
        return temp
    elif i is not k:
        if temp is None:
            temp = val[i]
        elif temp != val[i]:
            temp = val[i]
            counter = 0
        # print(f"temp = {temp} , counter = {counter} , i = {i} , k = {k} , median = {int((k/2)+1)}, val[i] = {val[i]}")
        return hitung(val,k,temp,i+1,counter+1)
    else:
        return None    

def Final(df,col,k):
    terdekat = []
    mean = 0
    minimum = None
    maximum = None
    fuzzy_val = []
    euc_val = []
    terdekat_append = terdekat.append
    fuzzy_append = fuzzy_val.append
    
    for val in range(k):
        terdekat_append(euc_dist[col][val])
    
    for x,y in terdekat:
        if (minimum is None or maximum is None):
            minimum = y
            maximum = y
        elif(y<minimum):
            minimum = y
        elif(y>maximum):
            maximum = y
        mean += y
        euc_val.append(y)
    mean /= len(terdekat)
    euc_val.sort()
    print(euc_val)
    
    fuzzy = hitung(euc_val,7)
    print("min : ",minimum)
    print("max : ",maximum)
    print("mean : ",mean)
    for x,y in terdekat:
        if fuzzy is None:
            fuzzy_append((y,fuzzy_triangular(y,minimum,mean,maximum)))
        else:
            fuzzy_val.append((y,fuzzy))
        
    print("fuzzy : ",fuzzy_val)
    print("fuzzy : ",fuzzy)
    
    pembilang = 0
    penyebut = 0 
    
    for nilai,bobot in fuzzy_val:
        pembilang += nilai*bobot
        penyebut += bobot
            
    print("pembilang : ", pembilang)
    print("penyebut : ",penyebut)
    
    if fuzzy is not None:
        tertimbang = fuzzy
    else:
        tertimbang = pembilang / penyebut
    
    print("rata-rata tertimbang : ",tertimbang)
    index = dummy.index[dummy.iloc[:,col] == 0].tolist()
    index = index[0]
    dummy.iloc[index,col] = tertimbang
    print(" ")

In [10]:
def fuzzy_triangular(val, minimum, mean, maximum):
   if val <= minimum or val >= maximum: 
        return 0.0
   elif minimum < val <= mean:
        return float((val - minimum)/(mean - minimum)) 
   elif mean < val < maximum:
        return float((maximum - val)/(maximum - mean))

In [11]:
df2 = df.copy()
dummy = df2.fillna(0)
dummy.fillna(0, inplace=True)
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.89,4.18,0.6,2.508,2.50,0.0,0.500,6.68,0.0,0.2,...,0.4,0.0,0.2,0.0,2.4,8.574,0.4,0.5,1.7148,1.0
1,3.89,0.00,0.0,0.000,4.83,0.0,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554,0.4,0.5,0.5108,0.0
2,3.89,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548,0.4,0.5,0.3096,0.0
3,3.89,0.00,0.2,0.000,10.80,0.6,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530,0.4,0.5,3.5060,1.0
4,3.89,0.00,0.2,0.000,0.08,0.2,0.016,0.08,5.0,0.2,...,0.4,0.0,0.0,0.0,0.0,1.416,0.0,0.0,0.2832,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,0.00,0.00,0.2,0.000,0.40,0.2,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578,0.4,0.5,0.3156,0.0
772,55.57,0.47,0.2,0.094,0.37,0.0,0.074,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568,0.4,0.5,0.0000,0.0
773,55.57,0.24,0.2,0.000,0.00,0.2,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456,0.4,0.5,0.2912,0.0
774,55.57,0.20,0.0,0.040,0.00,0.2,0.000,0.20,5.0,0.0,...,0.4,0.0,0.2,0.0,2.0,0.000,0.4,0.5,0.2880,0.0


In [12]:
count_zeros = np.count_nonzero(dummy == 0)

print("Jumlah angka 0 dalam dataset:", count_zeros)

Jumlah angka 0 dalam dataset: 5251


In [13]:
euc_dist = {}

In [14]:
for i in range(df2.shape[1]):
    print("Kolom :",i+1)
    current = dummy.index[dummy.iloc[:,i] == 0].tolist()
    for x in current:
        jarak(dummy,i)
        Final(dummy,i,7)

Kolom : 1
[2.72, 3.41, 3.89, 3.89, 3.89, 3.89, 3.89]
min :  2.72
max :  3.89
mean :  3.6542857142857144
fuzzy :  [(2.72, 0.0), (3.89, 0.0), (3.89, 0.0), (3.89, 0.0), (3.41, 0.7385321100917431), (3.89, 0.0), (3.89, 0.0)]
fuzzy :  None
pembilang :  2.5183944954128443
penyebut :  0.7385321100917431
rata-rata tertimbang :  3.41
 
[1.99, 2.72, 3.89, 55.57, 55.57, 55.57, 59.85]
min :  1.99
max :  59.85
mean :  33.59428571428571
fuzzy :  [(55.57, 0.16301213341313456), (55.57, 0.16301213341313456), (59.85, 0.0), (3.89, 0.0601184287845229), (2.72, 0.023098133164579857), (55.57, 0.16301213341313456), (1.99, 0.0)]
fuzzy :  None
pembilang :  27.472440371483117
penyebut :  0.5722529621885064
rata-rata tertimbang :  48.0075109902767
 
[1.85, 1.99, 2.37, 2.37, 2.37, 55.57, 55.57]
min :  1.85
max :  55.57
mean :  17.44142857142857
fuzzy :  [(2.37, 0.033351658420377496), (2.37, 0.033351658420377496), (55.57, 0.0), (1.99, 0.008979292651640089), (55.57, 0.0), (2.37, 0.033351658420377496), (1.85, 0.0)]
fu

[3.41, 3.41, 3.41, 3.41, 3.41, 3.41, 3.89]
median :  4
min :  3.41
max :  3.89
mean :  3.4785714285714286
fuzzy :  [(3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.41, 3.41), (3.89, 3.41)]
fuzzy :  3.41
pembilang :  83.0335
penyebut :  23.87
rata-rata tertimbang :  3.41
 
[1.85, 1.85, 1.99, 1.99, 1.99, 2.37, 3.89]
min :  1.85
max :  3.89
mean :  2.275714285714286
fuzzy :  [(1.99, 0.3288590604026843), (1.99, 0.3288590604026843), (1.85, 0.0), (1.99, 0.3288590604026843), (1.85, 0.0), (2.37, 0.9415929203539823), (3.89, 0.0)]
fuzzy :  None
pembilang :  4.194863811842963
penyebut :  1.9281701015620354
rata-rata tertimbang :  2.1755672948380695
 
[2.37, 2.72, 3.89, 55.57, 55.57, 55.57, 55.57]
min :  2.37
max :  55.57
mean :  33.037142857142854
fuzzy :  [(55.57, 0.0), (55.57, 0.0), (55.57, 0.0), (3.89, 0.049564447757022415), (55.57, 0.0), (2.72, 0.01141286625984069), (2.37, 0.0)]
fuzzy :  None
pembilang :  0.22384869800158388
penyebut :  0.060977314016863106
rata-rata 

[0.0, 0.0, 0.0, 1.85, 1.85, 1.85, 1.85]
median :  4
min :  0.0
max :  1.85
mean :  1.0571428571428572
fuzzy :  [(0.0, 1.85), (0.0, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85), (0.0, 1.85)]
fuzzy :  1.85
pembilang :  13.690000000000001
penyebut :  12.95
rata-rata tertimbang :  1.85
 
[0.0, 1.85, 1.85, 1.85, 1.85, 1.85, 1.85]
median :  4
min :  0.0
max :  1.85
mean :  1.5857142857142856
fuzzy :  [(0.0, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85), (1.85, 1.85)]
fuzzy :  1.85
pembilang :  20.535
penyebut :  12.95
rata-rata tertimbang :  1.85
 
[0.0, 1.85, 1.85, 1.99, 2.72, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  17.078571428571426
fuzzy :  [(0.0, 0.0), (1.99, 0.11652028439983272), (2.72, 0.15926390631534926), (55.57, 0.0), (55.57, 0.0), (1.85, 0.10832287745713093), (1.85, 0.10832287745713093)]
fuzzy :  None
pembilang :  1.0658678377248016
penyebut :  0.49242994562944387
rata-rata tertimbang :  2.1645065398335315
 
[0.0, 1.85, 1.85, 1.

[0.0, 0.0, 1.85, 2.37, 55.57, 55.57, 59.85]
min :  0.0
max :  59.85
mean :  25.03
fuzzy :  [(0.0, 0.0), (1.85, 0.07391130643228126), (55.57, 0.12291786329695581), (59.85, 0.0), (0.0, 0.0), (55.57, 0.12291786329695581), (2.37, 0.09468637634838194)]
fuzzy :  None
pembilang :  14.022233955669055
penyebut :  0.41443340937457485
rata-rata tertimbang :  33.83470936098065
 
[0.0, 1.99, 1.99, 2.34, 13.715065465094845, 21.61, 55.57]
min :  0.0
max :  55.57
mean :  13.887866495013549
fuzzy :  [(0.0, 0.0), (13.715065465094845, 0.987557409917445), (21.61, 0.814737566059025), (55.57, 0.0), (1.99, 0.1432905479552609), (1.99, 0.1432905479552609), (2.34, 0.1684924031232716)]
fuzzy :  None
pembilang :  32.11546193426319
penyebut :  2.2573684750102636
rata-rata tertimbang :  14.226947124402084
 
[0.0, 1.99, 1.99, 2.72, 55.57, 55.57, 55.57]
min :  0.0
max :  55.57
mean :  24.77285714285714
fuzzy :  [(0.0, 0.0), (55.57, 0.0), (1.99, 0.08032985410299291), (55.57, 0.0), (55.57, 0.0), (2.72, 0.10979758952770

[0.0, 0.0, 0.55, 0.63, 0.74, 0.77, 1.13]
min :  0.0
max :  1.13
mean :  0.5457142857142856
fuzzy :  [(0.74, 0.6674816625916868), (0.0, 0.0), (0.77, 0.6161369193154032), (0.0, 0.0), (1.13, 0.0), (0.63, 0.8557457212713935), (0.55, 0.9926650366748163)]
fuzzy :  None
pembilang :  2.0534474327628356
penyebut :  3.1320293398533
rata-rata tertimbang :  0.6556284153005464
 
[0.0, 0.0, 0.0, 0.43, 0.544423076923077, 0.77, 1.12]
min :  0.0
max :  1.12
mean :  0.40920329670329675
fuzzy :  [(0.544423076923077, 0.8097630734742782), (0.43, 0.9707416998415339), (0.77, 0.4924052100645462), (0.0, 0.0), (1.12, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  None
pembilang :  1.2374246467211143
penyebut :  2.2729099833803583
rata-rata tertimbang :  0.5444230769230769
 
[0.0, 0.0, 0.0, 0.53, 0.53, 1.64, 1.65]
min :  0.0
max :  1.65
mean :  0.6214285714285716
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (1.64, 0.009722222222222231), (1.65, 0.0), (0.53, 0.8528735632183907), (0.0, 0.0), (0.53, 0.8528735632183907)]
fuzzy :  None


...

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :  0.0
 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
median :  4
min :  0.0
max :  0.0
mean :  0.0
fuzzy :  [(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0)]
fuzzy :  0.0
pembilang :  0.0
penyebut :  0.0
rata-rata tertimbang :

In [15]:
dummy

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
0,3.890000,4.180000,0.600000,2.508000,2.50,0.200000,0.500,6.68,5.0,0.2,...,0.4,0.0,0.2,0.0,2.4,8.574000,0.4,0.5,1.714800,1.0
1,3.890000,0.786138,0.231579,0.137853,4.83,0.253659,0.966,4.83,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,2.554000,0.4,0.5,0.510800,0.0
2,3.890000,0.510000,0.200000,0.102000,0.23,0.200000,0.046,0.74,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.548000,0.4,0.5,0.309600,0.0
3,3.890000,4.576201,0.200000,1.517523,10.80,0.600000,6.480,10.80,6.0,0.6,...,0.4,0.0,0.2,0.0,4.4,17.530000,0.4,0.5,3.506000,1.0
4,3.890000,0.544423,0.200000,0.119320,0.08,0.200000,0.016,0.08,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.416000,0.4,0.5,0.283200,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,17.346655,0.000000,0.200000,0.000000,0.40,0.200000,0.080,0.89,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.578000,0.4,0.5,0.315600,0.0
772,55.570000,0.470000,0.200000,0.094000,0.37,0.200000,0.074,0.00,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.568000,0.4,0.5,0.295219,0.0
773,55.570000,0.240000,0.200000,0.000000,0.00,0.200000,0.008,0.28,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.456000,0.4,0.5,0.291200,0.0
774,55.570000,0.200000,0.200000,0.040000,0.00,0.200000,0.000,0.20,5.0,0.2,...,0.4,0.0,0.2,0.0,2.0,1.424132,0.4,0.5,0.288000,0.0


In [16]:
dummy.describe()

,Sector_score,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,Score_B.1,...,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
count,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,...,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000,776.000000
mean,19.529029,2.503013,0.349518,1.262761,8.538823,0.308626,5.950186,12.709222,5.064765,0.223158,...,0.509787,0.096649,0.216020,0.045361,2.694400,15.536441,0.551619,0.484536,6.493140,0.393041
std,23.823347,5.628074,0.173282,3.430660,21.591695,0.168032,29.988981,50.990686,0.254601,0.078932,...,0.275093,0.522712,0.065469,0.287016,0.852499,38.977649,0.432248,0.086617,37.084518,0.488741
min,1.850000,0.000000,0.200000,0.000000,0.000000,0.200000,0.000000,0.000000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.400000,0.000000,0.000000,0.280000,0.000000
25%,2.370000,0.378170,0.200000,0.000000,0.000000,0.200000,0.000000,0.420000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.000000,1.578000,0.400000,0.500000,0.315252,0.000000
50%,3.890000,0.925000,0.200000,0.147000,0.230000,0.200000,0.028000,1.285000,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,2.200000,2.200000,0.400000,0.500000,0.552200,0.000000
75%,55.570000,2.547500,0.600000,1.321500,2.477500,0.400000,0.702000,7.412500,5.000000,0.200000,...,0.400000,0.000000,0.200000,0.000000,3.200000,10.591000,0.400000,0.500000,3.295700,1.000000
max,59.850000,85.000000,0.600000,51.000000,144.390000,0.600000,758.778000,1268.910000,9.000000,0.600000,...,2.400000,9.000000,0.600000,5.400000,5.200000,583.688000,5.800000,0.500000,961.514400,1.000000


# Klasifikasi

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [18]:
x = dummy.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=0)

In [20]:
scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [21]:
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[120   4]
 [  6  64]]
0.9484536082474226


In [22]:
classifier = MLPClassifier(hidden_layer_sizes=3, learning_rate_init=0.1, max_iter=100)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)
cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print(cm)
print(accuracy)

[[124   0]
 [  5  65]]
0.9742268041237113
